In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as web

data_inicio = ''
data_final = ''
ticker = ''
codigo_bcb = 12 #cdi

#puxando dados do cdi do banco central 
url ='http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
df = pd.read_json(url)
df['data'] = pd.to_datetime(df['data'], dayfirst=True)
df.set_index('data', inplace=True)
df = df[(df.index >= data_inicio)&(df.index <= data_final)]

#acumulando as taxas de cdi
df['acumulado'] = (1 + df['valor'] / 100).cumprod()
df['acumulado'].iloc[0] = 1
#display(df)

#puxando cotacoes
cotacoes_ativo = web.DataReader('{}'.format(ticker), data_source='yahoo', start='{}'.format(data_inicio), end='{}'.format(data_final)) 
display(cotacoes_ativo)

retorno_ativo = pd.DataFrame()
retorno_ativo['Retornos Ativo'] = cotacoes_ativo['Adj Close'].pct_change()
retorno_ativo['Retorno Acumulado'] = (1 + retorno_ativo['Retornos Ativo']).cumprod() #retorno acumulado das cotacoes
retorno_ativo['Retorno Acumulado'].iloc[0] = 1 #retorno acumulado das cotacoes
display(retorno_ativo)

#calculo volatilidade
volatilidade_ativo = np.std(retorno_ativo['Retornos Ativo'])
volatilidade_ativo = volatilidade_ativo * np.sqrt(252)

#calculo indice sharpe
indice_sharpe = (retorno_ativo['Retorno Acumulado'][-1] - df['acumulado'][-1]) / volatilidade_ativo

print('Retorno Acumulado Ativo: {}'.format(retorno_ativo['Retorno Acumulado'][-1]))
print('Retorno Acumulado CDI: {}'.format(df['acumulado'][-1]))
print('Volatilidade Ativo: {}'.format(volatilidade_ativo))
print('Indice Sharpe: {}'.format(indice_sharpe))